In [8]:
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
except:
    print('Not running on colab')

import sys
import os
import numpy as np
import warnings

%cd /home/sasha/Public/Lab_Stuff/2016_ACCV_Acc_Anticipation/AnticipatingAccidentsCleaned
cur_dir = %pwd
sys.path.append(cur_dir)

nb_path = "/home/sasha/Public/Lab_Stuff/2016_ACCV_Acc_Anticipation/AnticipatingAccidentsCleaned/Accident_Prediction_Notebooks/utils/WC_appearance_4.py"
# !pip install --target=$nb_path cupy pynvrtc
sys.path.insert(0, nb_path)

## update root path if downloaded
root_path = '/home/sasha/Public/Lab_Stuff/2016_ACCV_Acc_Anticipation/AnticipatingAccidentsCleaned'

print(os.listdir('./'))
!pwd


# warnings.filterwarnings(action='once')
warnings.filterwarnings('ignore')
# ! pip install cupy pynvrtc


#### Attention Learning Blogs
# Pytorch attention implementations
# https://github.com/spro/practical-pytorch/blob/master/seq2seq-translation/seq2seq-translation.ipynb


Not running on colab
/home/sasha/Public/Lab_Stuff/2016_ACCV_Acc_Anticipation/AnticipatingAccidentsCleaned
['model', 'Fixed_Track_IDs', 'Accident_Prediction_Notebooks', 'Features', 'World_Coordinate_Extraction']
/home/sasha/Public/Lab_Stuff/2016_ACCV_Acc_Anticipation/AnticipatingAccidentsCleaned


In [11]:
import cv2
import argparse
import numpy as np
import os
import pdb
import time
import matplotlib.pyplot as plt
import sys
from itertools import permutations
from accident_physical_fgsm_lighter import evaluation #_2 as evaluation
# from qrnn import QRNN
import pickle 
import pandas as pd
from torchqrnn import QRNN

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# path
train_path = root_path + '/Features/original_features/training/' #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Change this line
test_path = root_path +'/Features/original_features/testing/'   #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Change this line
demo_path = root_path +'/Features/original_features/testing/'   #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Change this line

new_save_path = root_path + '/Features/physical_features/5_world_coordinates_5npz'

default_model_path = './model/WC_appearance_4_1/'   #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Change this line
save_path = './model/WC_appearance_4_1/'  #+ dir_str +'/'                   #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Change this line
video_path = './dataset/videos/testing/positive/'

# batch_number
train_num = 126
test_num = 46


############## Train Parameters #################
# Parameters
learning_rate = 0.0001
n_epochs = 110
batch_size = 10
display_step = 10

# Network Parameters
n_input = 4096 # fc6 or fc7(1*4096)          #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Change this line
n_detection = 20 # number of object of each image (include image features)

n_phys_hidden = 50
phys_dim = 50
n_img_hidden = 200 #20 + phys_dim # embedding image features 
n_att_hidden = 200 #20 + phys_dim # embedding object features
n_classes = 2 # has accident or not
n_frames = 100 # number of frame in each video 
n_hidden = n_img_hidden+n_att_hidden+n_phys_hidden*2 #300 #+25*2

p_drop = 0.5

num1 = (126, 46)


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
## change function that we draw from
from WC_appearance_4 import EmbedLayer, Attention_LSTM_2, Linear_Pred
from WC_appearance_4 import soft_max_entropy_loss_logits, temp_loss, run_batch, approx_evaluation, test_all

if torch.cuda.is_available():
  model1 = EmbedLayer(n_input, n_img_hidden, n_att_hidden, n_hidden, n_phys_hidden, p_drop).cuda()
  lstm = Attention_LSTM_2(batch_size,n_hidden,n_hidden, p_drop).cuda()
  linear_pred = Linear_Pred(batch_size,n_hidden,n_classes).cuda()
else:
  model1 = EmbedLayer(n_input, n_img_hidden, n_att_hidden, n_hidden, n_phys_hidden, p_drop)
  lstm = Attention_LSTM_2(batch_size,n_hidden,n_hidden, p_drop)
  linear_pred = Linear_Pred(batch_size,n_hidden,n_classes)

optimizer = torch.optim.Adam(list(model1.parameters())+list(lstm.parameters())+list(linear_pred.parameters()), lr = 0.00005)


In [13]:
len_dir = len(os.listdir(save_path))/3
ind = (len_dir-1)*3
tup = (ind, ind+1, ind+2)

if torch.cuda.is_available():
    map_location=lambda storage, loc: storage.cuda()
else:
    map_location='cpu'

map_location=torch.device('cpu')
model1.load_state_dict(torch.load(save_path + 'model1_85.pth', map_location = map_location))
lstm.load_state_dict(torch.load(save_path + 'model2_85.pth', map_location = map_location))
linear_pred.load_state_dict(torch.load(save_path + 'model3_85.pth', map_location = map_location  ))

saved_epoch = 86

Training Loop, Saving Every 15 Epochs

In [0]:
# approx_ATTC = 2.56565

saved_epoch = 86

num1 = (126, 46)
modes = ('training', 'testing')

approx_ATTC = 3.282
for epoch in range(saved_epoch, n_epochs):
    #  epoch = epoch 
     # random chose batch.npz
     epoch_loss = np.zeros((train_num,1),dtype = float)
     n_batchs = np.arange(1,train_num+1)
     np.random.shuffle(n_batchs)
     tStart_epoch = time.time()
    #  if epoch < 5:
    #  if True:
    #      approx_ATTC = 0

     count_batch = 0  
     start_time = time.time()
     for batch in n_batchs:
         count_batch += 1       

         file_name = '%03d' %batch
         batch_str = 'batch_'+file_name+'.npz'
        #  print(file_name)
         batch_data = np.load(train_path+batch_str, allow_pickle = True)
         batch_xs = batch_data['data']  #*0
         batch_ys = batch_data['labels']

        #  df_paths = retrieve_dir_custom(new_save_path, batch_data, 'training', train_num, split_type=2)
        #  phys_feat, feat_ind = retrieve_global_coord(df_paths, train_path, batch_str, train_num, mode='training')
         phys_dict = np.load(new_save_path+'training/'+batch_str)
         phys_feat, feat_ind = phys_dict['phys_feat'], phys_dict['feat_ind'] 
        #  phys_feat[:,:,:,[3,4,9,10,11,12]] = np.clip(phys_feat[:,:,:,[3,4,9,10,11,12]],-5,5)


         if np.sum(np.isnan(phys_feat)) != 0:
            print('NaNs in Physical Features')
            print(np.sum(np.isnan(phys_feat)))


         optimizer.zero_grad()
         
         if torch.cuda.is_available():
            batch_loss, all_alphas, soft_pred = run_batch(torch.from_numpy(batch_xs).float().cuda(), phys_feat, feat_ind, torch.from_numpy(batch_ys).float().cuda(), model1, lstm, linear_pred, approx_ATTC, True)
         else:
            batch_loss, all_alphas, soft_pred = run_batch(torch.from_numpy(batch_xs).float(),  phys_feat, feat_ind, torch.from_numpy(batch_ys).float(), model1, lstm, linear_pred, approx_ATTC, True)

         batch_loss.backward()
         optimizer.step()
         ## update steps
         epoch_loss[batch-1] = batch_loss.detach().cpu().numpy()/batch_size
          
         name = 'WC_1_'+str(epoch)
          
     #  if torch.cuda.is_available():
     #     approx_ATTC = test_all(train_num,train_path, phys_path_trn,model1, lstm, linear_pred, approx_ATTC, name, debug = True)
     #  else:
     #  approx_ATTC = test_all(test_num,test_path, new_save_path, ret_world_func, model1, lstm, linear_pred, approx_ATTC, name, 'testing', debug = True, log_file = '')
     #  if epoch > 5:
     approx_ATTC = test_all(train_num,train_path, new_save_path, new_save_path+'training/', model1, lstm, linear_pred, approx_ATTC, name, 'training', debug = True, log_file = '')
     approx_ATTC = test_all(test_num,test_path, new_save_path, new_save_path+'testing/', model1, lstm, linear_pred, approx_ATTC, name, 'testing', debug = True, log_file = '')
     #  approx_ATTC = 0  ##!!!!!!!!
        
     # print one epoch
     print( "Epoch:", epoch+1, " done. Loss:", np.mean(epoch_loss) )
     tStop_epoch = time.time()
     print( "Epoch Time Cost:", round(tStop_epoch - tStart_epoch,2), "s" )
     sys.stdout.flush()
     if True:
        model_name1, model_name2, model_name3 = 'model1_'+str(epoch), 'model2_'+str(epoch), 'model3_'+str(epoch)
        torch.save(model1.state_dict(),save_path + model_name1 + '.pth')
        torch.save(lstm.state_dict(),save_path + model_name2 + '.pth')
        torch.save(linear_pred.state_dict(),save_path + model_name3 + '.pth')
        print(' :::::: ')
        # if (epoch+1) % 10  == 0:
        #   print ("Training")
        #   #test_all(num,path,model1,model2,model3, ATTC, name, debug = False)
        #   test_all(train_num,train_path, new_save_path, new_save_path+'training/',model1, lstm, linear_pred, approx_ATTC, name, 'training', debug = False, log_file = '')
        print ("Testing")
        test_all(test_num,test_path, new_save_path, new_save_path+'testing/', model1, lstm, linear_pred, approx_ATTC, name, 'testing', debug = False, log_file = '')
        print(' :::::: ')

# epoch_loss[batch-1] = batch_loss.detach().cpu().numpy()/batch_size

Using features from path: /content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/features/training/
Average Precision= 0.7162 ,mean Time to accident= 3.108
Recall@80%, Time to accident= 3.356
Using features from path: /content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/features/testing/
Average Precision= 0.4128 ,mean Time to accident= 3.521
Recall@80%, Time to accident= 3.697
Epoch: 87  done. Loss: 0.5294804654187626
Epoch Time Cost: 1191.74 s
 :::::: 
Testing
Using features from path: /content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/features/testing/
Average Precision= 0.5435, Mean time to accident= 3.233, Adjusted AATC = 2.403
Recall@80%, Time to accident = 3.699
Recall@80%, Precision = 0.4045
 :::::: 
Using features from path: /content/gdrive/My Drive/Accident_Anticipation/Anticipating-Accidents/dataset/features/training/
Average Precision= 0.7314 ,mean Time to accident= 3.143
Recall@80%, Time to accident

KeyboardInterrupt: ignored

Training Loop, Saving Every 15 epochs


In [0]:
import time, datetime
# print('time elapsed from start ',datetime.timedelta(seconds=time.time()-start_time))
from pytz import timezone

# print('time elapsed from start ',datetime.timedelta(seconds=time.time()-start_time))

log_file = os.path.join(save_path, 'log.txt')
file2 = open(log_file, 'a')

fmt = "%Y-%m-%d %H:%M:%S %Z%z"
now_pacific = datetime.datetime.now(timezone('US/Pacific'))
now_pacific = now_pacific.strftime(fmt)

file2.write("\n" + now_pacific)
file2.close()

start_time = time.time()

### move window_length to better location
window_length = 20

for epoch in range(n_epochs):
#      epoch = epoch #+ 80
     # random chose batch.npz
     epoch_loss = np.zeros((train_num,1),dtype = float)
     n_batchs = np.arange(1,train_num+1)
     np.random.shuffle(n_batchs)
     tStart_epoch = time.time()
     if epoch == 0:
         approx_ATTC = 0
     for batch in n_batchs:
         file_name = '%03d' %batch
         batch_data = np.load(train_path+'batch_'+file_name+'.npz', allow_pickle = True)
         batch_xs = batch_data['data']
         batch_ys = batch_data['labels']
        #  if epoch < 15:
        #   batch_xs = np.zeros(batch_xs.shape)
          
         batch_data_phys = np.load(phys_path_trn+'batch_'+file_name+'.npz', allow_pickle = True)['data']
         ### shuffle last dimension in batch dimension, to reduce dependency on agent_id
         shuffle_phys_bool = True
         if shuffle_phys_bool:
           batch_data_phys = shuffle_ind(batch_data_phys) 

         batch_odom = retrieve_odometry_batch(batch_data['labels'], batch_data['ID'], 'training')
          

         ## retrieve batch_odometry

         optimizer.zero_grad()
         
         if torch.cuda.is_available():
            batch_loss, all_alphas, soft_pred = run_batch(torch.from_numpy(batch_xs).float().cuda(), torch.from_numpy(batch_data_phys).float().cuda(), torch.from_numpy(batch_odom).float().cuda(), torch.from_numpy(batch_ys).float().cuda(), model1, lstm, linear_pred, approx_ATTC, window_length, True)
         else:
            batch_loss, all_alphas, soft_pred = run_batch(torch.from_numpy(batch_xs).float(),torch.from_numpy(batch_data_phys).float(), torch.from_numpy(batch_odom).float() , torch.from_numpy(batch_ys).float(),model1, lstm, linear_pred, np.float(approx_ATTC), window_length, True)
         
         batch_loss.backward()
         optimizer.step()
         ## update steps
         epoch_loss[batch-1] = batch_loss.detach().cpu().numpy()/batch_size
          
         name = 'Perfect_Future_FC_'+str(epoch)
          
     if torch.cuda.is_available():
        approx_ATTC = test_all(train_num,train_path, phys_path_trn, 'training',model1, lstm, linear_pred, approx_ATTC, window_length, name, debug = True, log_file = log_file)
     else:
        approx_ATTC = test_all(train_num,train_path, phys_path_trn, 'training',model1, lstm, linear_pred, approx_ATTC, window_length, name, debug = True, log_file = log_file)

     tStop_epoch = time.time()
    


     
     # print( "Epoch Time Cost:", round(tStop_epoch - tStart_epoch,2), "s" )
     print('Epoch Time Cost ',datetime.timedelta(seconds=tStop_epoch - tStart_epoch))
     print('Time elapsed from start ',datetime.timedelta(seconds=time.time()-start_time))
     sys.stdout.flush()
     if (epoch+1) % 5  == 0:
        print('saving model\n')

        torch.save({
          'epoch': epoch,
          'model1_state_dict': model1.state_dict(),
          'lstm_state_dict': lstm.state_dict(),
          'linear_pred_state_dict': linear_pred.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'loss_history': loss_history, 
          'readme': 'Adding Physical Features to Local Appearance Features'
          }, os.path.join(save_path, 'chkpt-' + str(epoch) + '.tar'))
        
        
        ## retrieve odometry
        print(' :::::: ')
        print(' :::::: ')
        # print ("Training")
        # file2 = open(log_file, 'a')
        # file2.write("training\n")
        # file2.close()
        # test_all(train_num,train_path, phys_path_trn, 'training',model1, lstm, linear_pred, approx_ATTC, window_length, name, debug = True, log_file = log_file)
        print ("Testing")
        file2 = open(log_file, 'a')
        file2.write("testing\n")
        file2.close()
        test_all(test_num,test_path, phys_path_tst, 'testing',model1, lstm, linear_pred, approx_ATTC, window_length, name, debug = True, log_file = log_file)
        print(' :::::: ')
        print(' :::::: ')
     else:
        print ("Testing")
        file2 = open(log_file, 'a')
        file2.write("testing\n")
        file2.close()
        test_all(test_num,test_path, phys_path_tst, 'testing',model1, lstm, linear_pred, approx_ATTC, window_length, name, debug = True, log_file = log_file)

     # print one epoch
     str3 = "Epoch:" + str(epoch+1) + " done. Loss:" + str(np.mean(epoch_loss))
     print( str3 )
    
     print( "Epoch Time Cost:", round(tStop_epoch - tStart_epoch,2), "s" )
     sys.stdout.flush()

     loss_history.append(np.mean(epoch_loss))
     file2 = open(log_file, 'a')
     file2.write(str3 + "\n")
     file2.close()
        
    

NameError: ignored